# Envelope analysis

## Setup

In [ ]:
import sys
import os
import importlib

import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation
from matplotlib.lines import Line2D
import proplot as pplt

def ancestor_folder_path(current_path, ancestor_folder_name):  
    parent_path = os.path.dirname(current_path)
    if parent_path == current_path:
        raise ValueError("Couldn't find ancestor folder.")
    if parent_path.split('/')[-1] == ancestor_folder_name:
        return parent_path
    return ancestor_folder_path(parent_path, ancestor_folder_name)

sys.path.append(ancestor_folder_path(os.getcwd(), 'scdist'))
from tools import animation as myanim
from tools import ap_utils
from tools import beam_analysis as ba
from tools import plotting as myplt
from tools import plot_utils
from tools import utils
from tools.plotting import set_labels

Plot settings

In [ ]:
# Plotting
pplt.rc['figure.facecolor'] = 'white' 
pplt.rc['grid.alpha'] = 0.04
pplt.rc['axes.grid'] = False
pplt.rc['savefig.dpi'] = 'figure'
pplt.rc['animation.html'] = 'jshtml' 

plt_kws = dict(
    lw=1, 
    marker='.',
    markersize=3,
    legend=False,
)
format_kws = dict(
    xgrid=False,
    ygrid=True,
)
legend_kws = dict(
    framealpha=0.,
    ncols=1, 
    loc='r',
    handlelength=1.5,
)
animate = True
anim_kws = dict(skip=0, interval=1000./5.)

In [ ]:
def savefig(filename, folder, suffix='.png', **kws):
    kws.setdefault('dpi', 300)
    kws.setdefault('facecolor', 'white')
    filename = ''.join([filename, suffix])
    plt.savefig(os.path.join(folder, filename), **kws)

In [ ]:
width, height = 3.75, 2.5

def setup_figure(opt=1):
    """Convenience function to create subplots."""
    if opt == 1:
        nrows, ncols, figsize = 1, 1, (width, height)
    elif opt == 2:
        nrows, ncols, figsize = 3, 1, (width, 2*height)
    elif opt == 3:
        nrows, ncols, figsize = 3, 2, (1.6*width, 2*height)
    elif opt == 4:
        nrows, ncols, figsize = 1, 2, (7, 2.5)
    fig, axes = pplt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, spany=False, aligny=True)
    return fig, axes

Input/output file locations.

In [ ]:
# Input file locations
filenames = {
    'env_params': '_output/data/envelope/env_params.dat',
    'test_bunch_coords': '_output/data/envelope/test_bunch_coords.npy',
    'bunch_coords': '_output/data/bunch/bunch_coords.npy',
    'bunch_moments': '_output/data/bunch/bunch_moments.dat',
    'transfer_matrix': '_output/data/transfer_matrix.dat'
}
files_exist = {key: utils.file_exists(filename) for key, filename in filenames.items()}

# Output file locations
dirs = {
    'env': './_output/figures/envelope/',
    'bunch': './_output/figures/bunch/',
    'comparison': './_output/figures/comparison/'
}

## Envelope

In [ ]:
# Load correct x labels.
file = open('_output/data/xlabel.txt', 'r')
xlabel = file.readlines()[0]
file.close()
format_kws['xlabel'] = xlabel

# Load correct x values.
xvals = np.loadtxt('_output/data/xvals.dat')

print("xlabel = '{}'".format(xlabel))
print('xvals =')
print(xvals)

Compute beam statistics from the envelope parameters.

In [ ]:
mode = int(np.loadtxt('_output/data/mode.txt'))
env_params = np.loadtxt(filenames['env_params'])
env_params *= 1000. # [convert to mm mrad]
env_stats = ba.BeamStats(mode=mode)
env_stats.read_env(env_params)

### Twiss parameters 

#### 2D Twiss

In [ ]:
fig, axes = setup_figure(2);
g1 = axes[0].plot(xvals, env_stats.twiss2D[['beta_x','beta_y']], **plt_kws)
g2 = axes[1].plot(xvals, env_stats.twiss2D[['alpha_x','alpha_y']], **plt_kws)
g3 = axes[2].plot(xvals, env_stats.twiss2D[['eps_x','eps_y']], **plt_kws)
axes[0].set_ylabel('[m/rad]')
axes[1].set_ylabel('[rad]')
axes[2].set_ylabel('[mm mrad]')
axes[0].legend(g1, [r'$\beta_x$', r'$\beta_y$'], **legend_kws)
axes[1].legend(g2, [r'$\alpha_x$', r'$\alpha_y$'], **legend_kws)
axes[2].legend(g3, [r'$\varepsilon_x$', r'$\varepsilon_y$'], **legend_kws)
axes.format(**format_kws)
savefig('twiss2D', dirs['env'])

#### 4D Twiss 

In [ ]:
fig, axes = setup_figure(2);
g1 = axes[0].plot(xvals, env_stats.twiss4D[['beta_x','beta_y']], **plt_kws)
g2 = axes[1].plot(xvals, env_stats.twiss4D[['alpha_x','alpha_y']], **plt_kws)
g3 = axes[2].plot(xvals, env_stats.twiss4D['u'], color='black', **plt_kws)
axes[0].set_ylabel('[m/rad]')
axes[1].set_ylabel('[rad]')
subx = '{}x'.format(mode)
suby = '{}y'.format(mode)
axes[0].legend(g1, [r'$\beta_{}$'.format(subx), r'$\beta_{}$'.format(suby)], **legend_kws)
axes[1].legend(g2, [r'$\alpha_{}$'.format(subx), r'$\alpha_{}$'.format(suby)], **legend_kws)
axes[2].legend(g3, ['u'], **legend_kws)
axes.format(**format_kws)
savefig('twiss4D', dirs['env'])

#### Phase difference (nu)

In [ ]:
fig, ax = setup_figure(1)
ax.plot(xvals, env_stats.twiss4D['nu'], color='black', **plt_kws)
ax.format(ylabel=r'$\nu$', yformatter='deg', **format_kws)
savefig('twiss4D_nu', dirs['env'])

#### Emittances

In [ ]:
fig, ax = pplt.subplots(figsize=(4.0, 2.5))
g1 = ax.plot(xvals, env_stats.twiss2D['eps_x'], **plt_kws)
g2 = ax.plot(xvals, env_stats.twiss2D['eps_x'], **plt_kws)
g3 = ax.plot(xvals, env_stats.twiss4D['eps_1'], **plt_kws)
g4 = ax.plot(xvals, env_stats.twiss4D['eps_2'], **plt_kws)
ax.format(ylabel='[mm mrad]', **format_kws)
ax.legend([g1, g2, g3, g4],
          labels=[r'$\varepsilon_x$', r'$\varepsilon_y$', 
                  r'$\varepsilon_1$', r'$\varepsilon_2$'], 
          ncols=1, loc='r', framealpha=0.)
savefig('emittances', dirs['env'])

### Moments 

In [ ]:
fig, ax = setup_figure(1)
ax.plot(xvals, env_stats.moments[['x_rms', 'y_rms']], **plt_kws)
ax.format(ylabel='Beam size [mm]', **format_kws)
savefig('beamsize', dirs['env'])

In [ ]:
fig, ax = setup_figure(1)
ax.plot(xvals, env_stats.moments[['xp_rms', 'yp_rms']], **plt_kws)
ax.format(ylabel='Beam div. [mrad]', **format_kws)
savefig('beamdiv', dirs['env'])

In [ ]:
fig, axes = pplt.subplots(nrows=4, ncols=4, sharey=False, figsize=(8, 6), 
                          spany=False, aligny=True)
myplt.make_lower_triangular(axes)
axes.format(suptitle='Second-order moments', **format_kws)
for i in range(4):
    for j in range(i + 1):
        ax = axes[i, j]
        col = plot_utils.moment_label(i, j)
        ax.plot(xvals, env_stats.moments[col], color='black', **plt_kws)
        ax.set_title(plot_utils.moment_label_string(i, j))
set_labels(axes[0:, 0], [r'[mm$^2$]', r'[mm mrad]', r'[mm$^2$]', r'mm mrad'], 'ylabel')
set_labels(axes[1:, 1], [r'[mrad$^2$]', r'[mm mrad]', r'[mrad$^2$]'], 'ylabel')
set_labels(axes[2:, 2], [r'[mm$^2$]', r'[mm mrad]'], 'ylabel')
set_labels(axes[3:, 3], [r'[mrad$^2$]'], 'ylabel')
savefig('all_moments', dirs['env'])

In [ ]:
fig, axes = pplt.subplots(nrows=4, ncols=4, sharey=False, figsize=(8, 6), 
                          spany=False, aligny=True)
myplt.make_lower_triangular(axes)
axes.format(suptitle='Correlation coefficents', **format_kws)
for i in range(4):
    for j in range(i + 1):
        ax = axes[i, j]
        col = plot_utils.moment_label(i, j)
        ax.plot(xvals, env_stats.corr[col], color='black', **plt_kws)
        ax.set_title(plot_utils.moment_label_string(i, j))
savefig('all_correlations', dirs['env'])

### Real space orientation

In [ ]:
fig, axes = setup_figure(2)
axes[0].plot(xvals, env_stats.realspace['angle'], color='black', **plt_kws)
axes[1].plot(xvals, env_stats.realspace[['cx', 'cy']], color='black', **plt_kws)
axes[2].plot(xvals, env_stats.realspace['area'], color='black', **plt_kws)
set_labels(axes, ['tilt angle', 'ellipse axes [mm]', 'area [frac. change]'], 'ylabel')
axes.format(**format_kws)
axes[0].format(yformatter='deg')
savefig('beam_dims', dirs['env'])

### Tunes 

The tunes are computed for a particle on the beam envelope (the tune of every particle is the same). Of course this only makes sense if the data is turn-by-turn.

In [ ]:
env_coords = np.array([ba.get_ellipse_coords(p) for p in env_params])
fig, ax = pplt.subplots(figsize=(6, 2))
myplt.fft(ax, env_coords[:, 0, 0], env_coords[:, 0, 2])
ax.legend(ncols=1)
ax.format(**format_kws)
savefig('fft', dirs['env']);

### Phase space projections 

In [ ]:
frames = [0, len(env_params) - 1]
axes = myplt.corner_env(
    env_params[frames],
    figsize=(5, 5),
    limits=None,
    grid_kws=dict(constrained_layout=False),
    cmap=('red7', 'blue7'),
)
if 'Turn' in xlabel:
    fstr = 'Turn = {}'
elif 'Position' in xlabel:
    fstr = 'Position = {:.2f} [m]'
axes[1, 1].legend([fstr.format(frame) for frame in frames], 
                  loc=(0., 1.5), handlelength=1., framealpha=0.)
savefig('env_init_final', dirs['env'])

In [ ]:
if animate:
    if 'Turn' in xlabel:
        text_fmt = 'Turn = {}'
    elif 'Position' in xlabel:
        text_fmt = 'Position = {:.2f} [m]'
    anim = myanim.corner_env(
        env_params,
        dims='all',
        figsize=(5, 5),
        limits=None, 
        show_init=False, 
        plt_kws=dict(lw=0),
        text_fmt=text_fmt, text_vals=xvals,
        **anim_kws
    )
    utils.play(anim)

### Transfer matrix 

Keep in mind that the eigenvectors of the "effective lattice" (lattice + space charge) lose meaning if the beam is not matched.

In [ ]:
if files_exist['transfer_matrix']:
    M = np.loadtxt(filenames['transfer_matrix'])
    M_eigvals, M_eigvecs = np.linalg.eig(M)
    M_eigtunes = np.degrees(np.arccos(M_eigvals.real))
    print('M =')
    print(M)
    print()
    print('Eigenvalues:')
    print(M_eigvals[[0, 2]])
    print()
    print('Phase advances [deg]:')
    print(M_eigtunes[[0, 2]])
    print()
    print('Tunes:')
    print(M_eigtunes[[0, 2]] / 360.)

In [ ]:
if files_exist['transfer_matrix']:
    
    colors = ['red', 'blue']
    
    fig, axes = pplt.subplots(ncols=2, figsize=(5.25, 2.5), 
                              share=False, span=False)
    axes.format(grid=False)
    myplt.despine(axes)
    ax1, ax2 = axes

    # Plot eigenvalues in complex plane
    psi = np.linspace(0, 2*np.pi, 50)
    x_circ, y_circ = np.cos(psi), np.sin(psi)
    ax1.plot(x_circ, y_circ, 'k-', zorder=0, lw=0.5, alpha=0.5)
    for e, c in zip(M_eigvals, colors):
        ax1.scatter(e.real, e.imag, color=colors[i % 2])
    scale = 1.25
    ax1.format(
        xticks=[-1, -0.5, 0, 0.5, 1], yticks=[-1, -0.5, 0, 0.5, 1], 
        ylim=(-scale, scale), xlim=(-scale, scale),
        xlabel='Real', ylabel='Imag', title='Eigenvalues')
    ax1.annotate(r'$\mu_1 = {:.2f}\degree$'.format(M_eigtunes[0]), xy=(0, +0.1), horizontalalignment='center')
    ax1.annotate(r'$\mu_2 = {:.2f}\degree$'.format(M_eigtunes[2]), xy=(0, -0.1), horizontalalignment='center')

    # Plot turn-by-turn trajectory of eigenvectors.
    myplt.eigvec_trajectory(ax2, M, 'x', 'y', scatter_kws=dict(s=15, alpha=0.3, mec='none'))
    ax2.format(xticklabels=[], yticklabels=[], 
               ylabel='y', xlabel='x', title='Eigenvectors')

    # Add legend.
    lines = [Line2D([0], [0], color=c, lw=2) for c in colors]
    ax2.legend(lines, labels=[r'$\vec{v}_1$', r'$\vec{v}_2$'],
               loc=(1.05, 0.7), handlelength=1, ncols=1);

    savefig('eigvecs_realspace', dirs['env'])

In [ ]:
if files_exist['transfer_matrix']:
    
    # Set up figure
    fig, axes = pplt.subplots(nrows=3, ncols=3, figsize=(5, 5), span=False)
    axes.format(grid=False, suptitle='Transfer matrix eigenvectors')
    myplt.make_lower_triangular(axes)
    myplt.despine(axes)
    labels = ["x", "x'", "y", "y'"]
    set_labels(axes[-1, :], labels[:-1], 'xlabel')
    set_labels(axes[:, 0], labels[1:], 'ylabel')

    # Plot eigenvectors and their trajectories.
    for i in range(3):
        for j in range(3):
            if i >= j:
                ax = axes[i, j]
                dim1 = ['xp', 'y', 'yp'][i]
                dim2 = ['x', 'y', 'xp'][j]
                myplt.eigvec_trajectory(ax, M, dim1, dim2, colors=['red8', 'blue8'],
                                        scatter_kws=dict(s=9, alpha=0.4, mec='none'), 
                                        arrow_kws=dict(lw=1))

    # Zoom out a bit.
    for i in range(3):
        ymin, ymax = axes[i, 0].get_ylim()
        xmin, xmax = axes[-1, i].get_xlim()
        scale = 1.2
        axes[i, :].format(ylim=(-scale*ymax, scale*ymax))
        axes[:, i].format(xlim=(-scale*xmax, scale*xmax))

    savefig('eigvecs', dirs['env'])

### Test bunch

The linear space charge kicks for the test bunch are calculated directly from the envelope parameters.

In [ ]:
if files_exist['test_bunch_coords']:
    test_coords = np.load(filenames['test_bunch_coords'])
    nframes, ntestparts, ndims = test_coords.shape
    print('Test bunch coordinates:')
    print('nparts, nframes = {}, {}'.format(ntestparts, nframes))

## Bunch

In [ ]:
if files_exist['bunch_coords']:
    coords = np.load(filenames['bunch_coords'])  
    coords[:, :, :4] *= 1000. # convert to [mm mrad]
    n_frames, n_parts, n_dims = coords.shape
    print('Bunch coordinates:')
    print('n_frames, n_parts = {}, {}'.format(n_frames, n_parts))
        
if files_exist['bunch_moments']:
    moments = np.loadtxt(filenames['bunch_moments'])
    moments *= 1e6 # convert to [mm mrad]
    bunch_stats = ba.BeamStats(mode)
    bunch_stats.read_moments(moments)

### Twiss parameters 

#### 2D Twiss

In [ ]:
fig, axes = setup_figure(2);
g1 = axes[0].plot(xvals, bunch_stats.twiss2D[['beta_x','beta_y']], **plt_kws)
g2 = axes[1].plot(xvals, bunch_stats.twiss2D[['alpha_x','alpha_y']], **plt_kws)
g3 = axes[2].plot(xvals, bunch_stats.twiss2D[['eps_x','eps_y']], **plt_kws)
axes[0].set_ylabel('[m/rad]')
axes[1].set_ylabel('[rad]')
axes[2].set_ylabel('[mm mrad]')
axes[0].legend(g1, [r'$\beta_x$', r'$\beta_y$'], **legend_kws)
axes[1].legend(g2, [r'$\alpha_x$', r'$\alpha_y$'], **legend_kws)
axes[2].legend(g3, [r'$\varepsilon_x$', r'$\varepsilon_y$'], **legend_kws)
axes.format(**format_kws)
savefig('twiss2D', dirs['bunch'])

#### Emittance 

In [ ]:
if files_exist['bunch_moments']:
    fig, ax = pplt.subplots(figsize=(4.0, 2.5))
    g1 = ax.plot(xvals, bunch_stats.twiss2D['eps_x'], **plt_kws)
    g2 = ax.plot(xvals, bunch_stats.twiss2D['eps_x'], **plt_kws)
    g3 = ax.plot(xvals, bunch_stats.twiss4D['eps_1'], **plt_kws)
    g4 = ax.plot(xvals, bunch_stats.twiss4D['eps_2'], **plt_kws)
    ax.format(ylabel='[mm mrad]', **format_kws)
    ax.legend([g1, g2, g3, g4],
              labels=[r'$\varepsilon_x$', r'$\varepsilon_y$', 
                      r'$\varepsilon_1$', r'$\varepsilon_2$'], 
              ncols=1, loc='r', framealpha=0.)
    savefig('emittances', dirs['bunch'])

### Moments 

In [ ]:
if files_exist['bunch_moments']:
    fig, ax = setup_figure(1)
    ax.plot(xvals, bunch_stats.moments[['x_rms', 'y_rms']], **plt_kws)
    ax.format(ylabel='Beam size [mm]', **format_kws)
    savefig('beamsize', dirs['bunch'])

In [ ]:
if files_exist['bunch_moments']:
    fig, ax = setup_figure(1)
    ax.plot(xvals, bunch_stats.moments[['xp_rms', 'yp_rms']], **plt_kws)
    ax.format(ylabel='Beam div. [mrad]', **format_kws)
    savefig('beamdiv', dirs['bunch'])

In [ ]:
fig, axes = pplt.subplots(nrows=4, ncols=4, sharey=False, figsize=(8, 6), 
                          spany=False, aligny=True)
myplt.make_lower_triangular(axes)
axes.format(suptitle='Second-order moments', **format_kws)
for i in range(4):
    for j in range(i + 1):
        ax = axes[i, j]
        col = plot_utils.moment_label(i, j)
        ax.plot(xvals, bunch_stats.moments[col], color='black', **plt_kws)
        ax.set_title(plot_utils.moment_label_string(i, j))
set_labels(axes[0:, 0], [r'[mm$^2$]', r'[mm mrad]', r'[mm$^2$]', r'mm mrad'], 'ylabel')
set_labels(axes[1:, 1], [r'[mrad$^2$]', r'[mm mrad]', r'[mrad$^2$]'], 'ylabel')
set_labels(axes[2:, 2], [r'[mm$^2$]', r'[mm mrad]'], 'ylabel')
set_labels(axes[3:, 3], [r'[mrad$^2$]'], 'ylabel')
savefig('all_moments', dirs['bunch'])

In [ ]:
fig, axes = pplt.subplots(nrows=4, ncols=4, sharey=False, figsize=(8, 6), 
                          spany=False, aligny=True)
myplt.make_lower_triangular(axes)
axes.format(suptitle='Correlation coefficents', **format_kws)
for i in range(4):
    for j in range(i + 1):
        ax = axes[i, j]
        col = plot_utils.moment_label(i, j)
        ax.plot(xvals, bunch_stats.corr[col], color='black', **plt_kws)
        ax.set_title(plot_utils.moment_label_string(i, j))
savefig('all_correlations', dirs['bunch'])

### Beam orientation

In [ ]:
if files_exist['bunch_moments']:
    fig, axes = setup_figure(2)
    axes[0].plot(xvals, bunch_stats.realspace['angle'], color='black', **plt_kws)
    axes[1].plot(xvals, bunch_stats.realspace[['cx', 'cy']], color='black', **plt_kws)
    axes[2].plot(xvals, bunch_stats.realspace['area'], color='black', **plt_kws)
    set_labels(axes, ['tilt angle', 'ellipse axes [mm]', 'area [frac. change]'], 'ylabel')
    axes.format(**format_kws)
    axes[0].format(yformatter='deg')
    savefig('beam_dims', dirs['bunch'])

### Tunes 

In [ ]:
fig, ax = pplt.subplots(figsize=(6, 2))
part_index = 0
myplt.fft(ax, coords[:, part_index, 0], env_coords[:, part_index, 2])
ax.legend(ncols=1)
ax.format(**format_kws)
savefig('fft', dirs['bunch']);

### Phase space projections 

In [ ]:
figsize = (5.5, 5.5)
kind = 'scatter'
autolim_kws = dict(sigma=3.0)
samples = 10000
plot_kws = dict(color='black', ms=1)

In [ ]:
if files_exist['bunch_coords']:
    frames = [0, len(coords) - 1]
    for frame in frames:
        axes = myplt.corner(
            coords[i], 
            figsize=figsize,
            kind=kind,
            samples=samples,
            autolim_kws=autolim_kws,
            **plot_kws
        )
        plt.suptitle(text_fmt.format(xvals[frame]))
        savefig('bunch_frame{}'.format(frame), dirs['bunch'])
        plt.show()

In [ ]:
if files_exist['bunch_coords'] and animate:
    anim = myanim.corner(
        coords, 
        text_fmt=text_fmt, text_vals=xvals,
        figsize=figsize,
        kind=kind,
        samples=samples,
        autolim_kws=autolim_kws,
        plot_kws=plot_kws,
        **anim_kws
    )
    utils.play(anim)

## Comparison 

In [ ]:
if files_exist['bunch_moments']:
    plt_kws_env = dict(
        lw=None,
        marker=None,
        markersize=None,
        color='black',
        legend=False,
    )
    plt_kws_bunch = dict(
        lw=0,
        marker='.',
        markersize=None,
        color='red',
        legend=False,
    )
    dataframes = [env_stats, bunch_stats]
    kws_list = [plt_kws_env, plt_kws_bunch]

### Moments

In [ ]:
if files_exist['bunch_moments']:
    fig, axes = setup_figure(4)
    for ax, key in zip(axes, ('x_rms', 'y_rms')):
        for df, kws in zip(dataframes, kws_list):
            ax.plot(xvals, df.moments[key], **kws)
    axes.format(xlabel='Turn number', ylabel='[mm]')
    set_labels(axes, [r'$\sqrt{\langle{x^2}\rangle}$', r'$\sqrt{\langle{y^2}\rangle}$'], 'title')
    axes[1].legend(labels=['theory', 'calc'], ncols=1, loc=(1.02, 0), fontsize='small')
    axes.format(**format_kws)
    savefig('beamsize', dirs['comparison'])

In [ ]:
if files_exist['bunch_moments']:
    fig, axes = setup_figure(4)
    for ax, key in zip(axes, ('xp_rms', 'yp_rms')):
        for df, kws in zip(dataframes, kws_list):
            ax.plot(xvals, df.moments[key], **kws)
    axes.format(xlabel='Turn number', ylabel='[mrad]')
    set_labels(axes, [r"$\sqrt{\langle{x'^2}\rangle}$", r"$\sqrt{\langle{y'^2}\rangle}$"], 'title')
    axes[1].legend(labels=['theory', 'calc'], ncols=1, loc=(1.02, 0), fontsize='small')
    axes.format(**format_kws)
    savefig('up', dirs['comparison'])

In [ ]:
if files_exist['bunch_moments']:
    fig, ax = pplt.subplots(figsize=(1.25*width, height))
    for df, kws in zip(dataframes, kws_list):
        ax.plot(xvals, df.corr['xy'], **kws)
    ax.format(title=r"$x$-$y$ corr. coef.", xlabel='Turn number')
    ax.legend(labels=['theory', 'calc'], ncols=1, loc=(1.02, 0), fontsize='small')
    ax.format(**format_kws)
    savefig('xy_corr', dirs['comparison'])

In [ ]:
if files_exist['bunch_moments']:
    fig, axes = pplt.subplots(nrows=4, ncols=4, sharey=False, figsize=(8, 6), spany=False, aligny=True)
    myplt.make_lower_triangular(axes)
    for i in range(4):
        for j in range(i + 1):
            ax = axes[i, j]
            ax.format(title=plot_utils.moment_label_string(i, j))
            col = plot_utils.moment_label(i, j)
            for df, kws in zip(dataframes, kws_list):
                ax.plot(xvals, df.moments[col].values, **kws)
    set_labels(axes[0:, 0], [r'[mm$^2$]', r'[mm$\cdot$mrad]', r'[mm$^2$]', r'mm$\cdot$mrad'], 'ylabel')
    set_labels(axes[1:, 1], [r'[mrad$^2$]', r'[mm$\cdot$mrad]', r'[mrad$^2$]'], 'ylabel')
    set_labels(axes[2:, 2], [r'[mm$^2$]', r'[mm$\cdot$mrad]'], 'ylabel')
    set_labels(axes[3:, 3], [r'[mrad$^2$]'], 'ylabel')
    custom_lines = [Line2D([0], [0], color=plt_kws_env['color']),
                    Line2D([0], [0], color=plt_kws_bunch['color'])]
    axes[0, 1].legend(custom_lines, ['theory', 'calc'], ncols=1)
    axes.format(suptitle='Transverse moments', **format_kws)
    savefig('all_moments', dirs['comparison'])

In [ ]:
if files_exist['bunch_moments']:
    fig, axes = pplt.subplots(nrows=4, ncols=4, sharey=False, figsize=(8, 6), spany=False, aligny=True)
    myplt.make_lower_triangular(axes)
    for i in range(4):
        for j in range(i + 1):
            ax = axes[i, j]
            ax.format(title=plot_utils.moment_label_string(i, j))
            col = plot_utils.moment_label(i, j)
            for df, kws in zip(dataframes, kws_list):
                ax.plot(xvals, df.corr[col].values, **kws)
    custom_lines = [Line2D([0], [0], color=plt_kws_env['color']),
                    Line2D([0], [0], color=plt_kws_bunch['color'])]
    axes[0, 1].legend(custom_lines, ['theory', 'calc'], ncols=1)
    axes.format(xlabel='Turn number', suptitle='Transverse moments', grid=False)
    savefig('all_corr', dirs['comparison'])

### Twiss parameters

In [ ]:
if files_exist['bunch_moments']:
    fig, axes = setup_figure(3)
    for df, kws in zip(dataframes, kws_list):
        axes[0, 0].plot(xvals, df.twiss2D['beta_x'].values, **kws)
        axes[0, 1].plot(xvals, df.twiss2D['beta_y'].values, **kws)
        axes[1, 0].plot(xvals, df.twiss2D['alpha_x'].values, **kws)
        axes[1, 1].plot(xvals, df.twiss2D['alpha_y'].values, **kws)
        axes[2, 0].plot(xvals, df.twiss2D['eps_x'].values, **kws)
        axes[2, 1].plot(xvals, df.twiss2D['eps_y'].values, **kws)
    axes.format(collabels=['Horizontal', 'Vertical'], **format_kws)
    axes[0, 0].set_ylabel(r'$\beta$ [m]')
    axes[1, 0].set_ylabel(r'$\alpha$ [m]')
    axes[2, 0].set_ylabel(r'$\varepsilon$ [m]')
    savefig('twiss2D', dirs['comparison'])

In [ ]:
if files_exist['bunch_moments']:
    fig, axes = setup_figure(3)
    for df, kws in zip(dataframes, kws_list):
        axes[0, 0].plot(xvals, df.twiss4D['beta_x'].values, **kws)
        axes[0, 1].plot(xvals, df.twiss4D['beta_y'].values, **kws)
        axes[1, 0].plot(xvals, df.twiss4D['alpha_x'].values, **kws)
        axes[1, 1].plot(xvals, df.twiss4D['alpha_y'].values, **kws)
        axes[2, 0].plot(xvals, df.twiss4D['u'].values, **kws)
    axes.format(collabels=['Horizontal', 'Vertical'], **format_kws)
    set_labels(axes[:, 0], [r'$\beta$ [m]', r'$\alpha$ [rad]', 'u'], 'ylabel')
    savefig('twiss4D', dirs['comparison'])

### Beam dimensions 

In [ ]:
if files_exist['bunch_moments']:
    fig, axes = pplt.subplots(nrows=2, figsize=(width, 1.5*height), spany=False, aligny=True)
    for df, kws in zip(dataframes, kws_list):
        axes[0].plot(xvals, df.realspace['angle'].values, **kws)
        axes[1].plot(xvals, df.realspace['area'].values, **kws)
    axes[0].format(ylabel='tilt angle', yformatter='deg')
    axes[1].format(ylabel=r'area [mm$^2$]')
    axes.format(**format_kws)
    savefig('beam_dims', dirs['comparison'])

### Phase space projections

In [ ]:
if files_exist['bunch_coords']:
    frames = [0, len(coords) - 1]
    for frame in frames:
        axes = myplt.corner(
            coords[frame], 
            figsize=figsize,
            kind=kind,
            samples=samples,
            autolim_kws=autolim_kws,
            env_params=env_params[frame],
            env_kws=dict(color='red', lw=0.5, zorder=int(1e12)),
            **plot_kws
        )
        plt.suptitle(text_fmt.format(xvals[frame]))
        savefig('bunch_frame{}'.format(frame), dirs['comparison'])
        plt.show()

In [ ]:
if files_exist['bunch_coords'] and animate:
    anim = myanim.corner(
        coords, 
        text_fmt=text_fmt, text_vals=xvals,
        figsize=figsize,
        kind=kind,
        samples=samples,
        autolim_kws=autolim_kws,
        plot_kws=plot_kws,
        env_params = env_params,
        env_kws=dict(color='red', lw=1.0, zorder=int(1e12)),
        **anim_kws
    )
    utils.play(anim)